In [2]:
import pandas as pd

matches = pd.read_csv("matches2223.csv", index_col=0)
odds = pd.read_csv(r"C:\Users\krawc\OneDrive\Documents\code\pythons\predictor\data\match_w_odds_1923.csv", index_col=[0,1,2])
ranked = pd.read_csv(r"C:\Users\krawc\OneDrive\Documents\code\pythons\predictor\data\ranked_mets_2223back.csv", index_col=[0])

matches

,team,date,time,round,day,venue,result,gf,ga,opponent,...,formation,referee,match_report,notes,sh,sot,dist,fk,pk,pkatt
0,Arsenal,2022-08-05,20:00:00,Matchweek 1,Fri,Away,W,2,0,Crystal Palace,...,4-3-3,Anthony Taylor,/en/matches/e62f6e78/Crystal-Palace-Arsenal-Au...,NaN,10,2,14.6,1,0,0
1,Arsenal,2022-08-13,15:00:00,Matchweek 2,Sat,Home,W,4,2,Leicester City,...,4-3-3,Darren England,/en/matches/7483b97f/Arsenal-Leicester-City-Au...,NaN,19,7,13.0,0,0,0
2,Arsenal,2022-08-20,17:30:00,Matchweek 3,Sat,Away,W,3,0,Bournemouth,...,4-3-3,Craig Pawson,/en/matches/7f11dd9e/Bournemouth-Arsenal-Augus...,NaN,14,6,14.8,0,0,0
3,Arsenal,2022-08-27,17:30:00,Matchweek 4,Sat,Home,W,2,1,Fulham,...,4-3-3,Jarred Gillett,/en/matches/8b69fd2d/Arsenal-Fulham-August-27-...,NaN,22,8,15.5,1,0,0
4,Arsenal,2022-08-31,19:30:00,Matchweek 5,Wed,Home,W,2,1,Aston Villa,...,4-3-3,Robert Jones,/en/matches/cc235aad/Arsenal-Aston-Villa-Augus...,NaN,22,8,16.3,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
755,Wolves,2023-04-29,15:00:00,Matchweek 34,Sat,Away,L,0,6,Brighton,...,4-4-2,David Coote,/en/matches/21f920e0/Brighton-and-Hove-Albion-...,NaN,10,2,18.7,0,0,0
756,Wolves,2023-05-06,15:00:00,Matchweek 35,Sat,Home,W,1,0,Aston Villa,...,4-4-2,Stuart Attwell,/en/matches/217a7faf/Wolverhampton-Wanderers-A...,NaN,6,2,17.6,0,0,0
757,Wolves,2023-05-13,15:00:00,Matchweek 36,Sat,Away,L,0,2,Manchester Utd,...,4-4-2,John Brooks,/en/matches/f5d61382/Manchester-United-Wolverh...,NaN,5,0,10.5,0,0,0
758,Wolves,2023-05-20,15:00:00,Matchweek 37,Sat,Home,D,1,1,Everton,...,4-4-2,David Coote,/en/matches/ff2b58c3/Wolverhampton-Wanderers-E...,NaN,13,5,17.9,1,0,0


In [3]:
def add_key_col(hist):
    ew_game=hist['date'] +' ' + hist['team']+'-' + hist['opponent']
    # ew_game=hist['date'].dt.date.astype('string') +' ' + hist['home_team']+'-' + hist['away_team']
    return ew_game

def prep_matches(matches):
    # matches=matches.drop('notes',axis=1)
    matches=matches.iloc[:,:10]
    # matches["date"] = pd.to_datetime(matches["date"])
    matches["target"] = (matches["result"] == "W").astype("int")
    matches["venue_code"] = matches["venue"].astype("category").cat.codes
    # matches["opp_code"] = matches["opponent"].astype("category").cat.codes
    # matches["hour"] = matches["time"].str.replace(":.+", "", regex=True).astype("int")
    # matches["day_code"] = matches["date"].dt.dayofweek
    return matches

def eliminate_outliers(matches,odd_cols,trsh=0.25):
    c1=matches.target==1
    c2=matches.pAvgH<trsh
    matches=matches[~(c1&c2)]#['pAvgH'].quantile(.2)
    #remove odds
    # matches=matches.iloc[:,:-6]
    matches=matches.drop(odd_cols,axis=1)
    return matches

def keysplit(text):
    sd,st=text.split(' ',1)
    sh,sa=st.split('-',1)
    return sd,sh,sa

def reverse_key(oldkey):
    sd,sh,sa = keysplit(oldkey)
    return sd + ' ' + sa + '-' + sh

def reverse_odds_df(odds2):
    odds3=odds2.copy()
    odds3.index=odds3.index.map(reverse_key)
    odds3['AvgH'],odds3['AvgA']=odds3['AvgA'],odds3['AvgH']
    odds3['pAvgH'],odds3['pAvgA']=odds3['pAvgA'],odds3['pAvgH']
    return odds3

def prep_odds(odds):
    cols=['AvgH', 'AvgD', 'AvgA','pAvgH','pAvgD','pAvgA']
    odds2=odds.droplevel([0,1]).loc[:,cols] 

    odds3=reverse_odds_df(odds2)
    
    odds23=pd.concat([odds2,odds3])
    return odds23

def merge_matches_odds(matches,odds):
    matches['key']=add_key_col(matches)
    matches=matches.set_index('key')
    
    matches2=matches.join(odds).reset_index(drop=True)
    odd_cols=list(odds.columns)
    return matches2,odd_cols

def rolling_averages(group, cols, new_cols,window=3):
    group = group.sort_values("date")
    rolling_stats = group[cols].rolling(window, closed='left').mean()
    group[new_cols] = rolling_stats
    group = group.dropna(subset=new_cols)
    return group
    # # EXAMPLE
    # grouped_matches = matches.groupby("team")
    # group = grouped_matches.get_group("Manchester City").sort_values("date")
    # rolling_averages(group, cols, new_cols)

def add_rolling(matches):
    cols = ['points']
    new_cols = [f"{c}_rolling" for c in cols]

    wdl={'W':3,'D':1,'L':0}
    matches['points']=matches['result'].map(wdl)

    matches_rolling = matches.groupby("team").apply(lambda x: rolling_averages(x, cols, new_cols))
    matches_rolling = matches_rolling.droplevel('team')
    matches_rolling = matches_rolling.reset_index(drop=True)

    matches_rolling = matches_rolling.drop('points',axis=1)
    return matches_rolling 

def merge_matches_metrics(matches,ranked):
    full=matches.set_index('team').join(ranked).reset_index(drop=False,names='team')
    suf='_opo'
    full=full.set_index('opponent').join(ranked,rsuffix=suf).reset_index(drop=False,names='opponent')
    team_cols=list(ranked.columns)
    opo_cols=[x+suf for x in team_cols]
    return full,team_cols,opo_cols

def calc_diffs(full,team_cols,opo_cols):
    # rankedlen=17
    # startcol=12+0
    
    # sc=startcol
    # rl=rankedlen

    # team_cols=full.columns[sc:sc+rl]
    # opo_cols=full.columns[sc+rl:sc+rl+rl]
    new_names=[x +'_diff' for x in team_cols]
    
    diff_vals=full[team_cols].values-full[opo_cols].values
    diff_df=pd.DataFrame(diff_vals,index=full[team_cols].index,columns=new_names)

    full=full.join(diff_df)
    return full

In [4]:
matches=prep_matches(matches)
odds=prep_odds(odds)


In [5]:
matches2,odd_cols=merge_matches_odds(matches,odds)

full,team_cols,opo_cols=merge_matches_metrics(matches2,ranked)
full=calc_diffs(full,team_cols,opo_cols)
# full=add_rolling(full)

full_w_outs=full.copy()
full=eliminate_outliers(full,odd_cols)

# full['r_ad']=full['atks']/full['defs_opo']
# full['r_err']=full['offsucc']/full['deferror_opo']

In [6]:
full_w_outs.to_csv("full_w_outs.csv")
full.to_csv("full.csv")